In [ ]:
import os
import sys
from pathlib import Path

ASL = os.path.join(str(Path.home()), "ASL")
src = os.path.join(str(Path.home()), "ASL", "src")
os.chdir( ASL )
sys.path.append(ASL)
sys.path.append(src)

from lightning import Network
from utils_asl import load_yaml
from visu import Visualizer

textwidth = 12.7

import matplotlib
import matplotlib.pyplot as plt
import skimage
from PIL import Image, ImageDraw
import imageio 
import numpy as np
import torch
import cv2
import pandas
from visu import *
from datasets_asl import MLHypersim
from datasets_asl import ScanNet

from utils_asl import load_yaml
from task import get_task_generator
from datasets_asl import adapter_tg_to_dataloader

In [ ]:
# 1 Step: Load config
# 2 Load: Dataloaders
# 3 Load: Model from each checkpoint 
# 4 Inference and plot
# 5 Creates a new folder in the ceckpoint path called visu_report and puts images there. 
checkpoint_path = "/home/jonfrey/Results/semi_4_scenes/2021-07-28T21:02:55_cl_005_unltimited_mem_ratio05"
show_in_jupyter = True

exp_path = [str(s) for s in Path(checkpoint_path).rglob("*.yml") if str(s).find("euler") == -1 ][0]
env_path = os.path.join("cfg/env", os.environ["ENV_WORKSTATION_NAME"] + ".yml")

visu_report = os.path.join(checkpoint_path,"visu_report")

vi = Visualizer(p_visu=visu_report, store=True)

exp = load_yaml( exp_path )
env = load_yaml( env_path )
tg = get_task_generator(
    name=exp["task_generator"].get("name", "scannet"),
    mode=exp["task_generator"]["mode"], 
    cfg=exp["task_generator"]["cfg"],
)
task_nr = 4
# Reinitalizing of all datasets
train_dataloader, val_dataloaders, task_name = adapter_tg_to_dataloader(
    tg, task_nr, exp["loader"], exp["replay"]["cfg_ensemble"], env
)

ckpt_paths = exp_path = [str(s) for s in Path(checkpoint_path).rglob("*.ckpt") if str(s).find("last") == -1]
ckpt_paths.sort()
ckpt_paths



In [ ]:
print("Exp Path: \n", exp_path , "\n\n")
print("Found Checkpoints: \n", ckpt_paths, "\n\n")
print(str(tg))

In [ ]:
from pseudo_label import FastSCNNHelperTorch
from utils_asl import load_yaml
frames = [0, 10, 20, 30]

def parse_batch(batch):
    ba = {}
    if len(batch) == 1:
      raise Exception("Dataloader is set to unique and not implemented")
    ba["images"] = batch[0]
    ba["label"] = batch[1]
    print(len( batch ))
    if len(batch) == 2:
      ba["ori_img"] = batch[2]
    if len(batch) == 4:
      ba["aux_label"] = batch[2]
      ba["aux_valid"] = batch[3]
    if len(batch) == 6:
      ba["aux_label"] = batch[3]
      ba["aux_valid"] = batch[4]
      ba["ori_img"] = batch[5]
    return ba
train_dataloader.visu=True
for kk, ckpt in enumerate( ckpt_paths[1:] ):
    exp["checkpoint_load"] = ckpt
    scnn = FastSCNNHelperTorch(device="cpu", exp=exp )
    print(ckpt)
    for i in range( len(train_dataloader) ):
            ba = parse_batch( train_dataloader.dataset[i])
            print(ba.keys())
            image = ba["images"][None]
            img =  ba["ori_img"]

            out = scnn.get_label_prob( image )
            la = torch.argmax( out,dim= 0)
            vi.epoch = i
            vi.plot_detectron( img , la , alpha=0.85, 
                                    text_off =True,
                                    draw_bound=False, 
                                    jupyter=show_in_jupyter, tag = "pred")

            vi.plot_detectron(  img , ba["label"]+1 , alpha=0.85, 
                                    text_off =True,
                                        draw_bound=False, 
                                        jupyter=show_in_jupyter, tag = "gt")
            try:
                vi.plot_detectron(  img , ba["aux_label"]+1 , alpha=0.85, 
                                        text_off =True,
                                            draw_bound=False, 
                                            jupyter=show_in_jupyter, tag = "aux")
            except:
                pass
    break

In [ ]:
from pseudo_label import FastSCNNHelperTorch
from utils_asl import load_yaml
frames = [0, 10, 20, 30]

def parse_batch(batch):
    ba = {}
    if len(batch) == 1:
      raise Exception("Dataloader is set to unique and not implemented")
    ba["images"] = batch[0]
    ba["label"] = batch[1]

    if len(batch) == 3:
      ba["ori_img"] = batch[2]
    if len(batch) == 4:
      ba["aux_label"] = batch[2]
      ba["aux_valid"] = batch[3]
    if len(batch) == 5:
      ba["aux_label"] = batch[2]
      ba["aux_valid"] = batch[3]
      ba["ori_img"] = batch[4]
    return ba
def plot_seq(seq):
    import matplotlib.pyplot as plt
    from mpl_toolkits.axes_grid1 import ImageGrid
    import numpy as np


    fig = plt.figure(figsize=(10, 40))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(1, 4),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, seq):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.axis('off')
    plt.show()
ref_pred = []
for kk, ckpt in enumerate( ckpt_paths ):
    exp["checkpoint_load"] = ckpt
    scnn = FastSCNNHelperTorch(device="cpu", exp=exp )
    print(ckpt)
    for i in range( len(val_dataloaders) ):
        seq = []
        gts = []
        dets = []
        a = []
        for f in frames:
            ba = parse_batch( val_dataloaders[i].dataset[f] )
            image = ba["images"][None]
            img =  ba["ori_img"]

            out = scnn.get_label_prob( image )
            la = torch.argmax( out,dim= 0)
            name = ckpt.split("/")[-1] + f"_vaidation_{i}_frame_{f}"
            
            seq.append( vi.plot_detectron( img , la , alpha=0.85, 
                                    text_off =True,
                                    draw_bound=False, 
                                    jupyter=show_in_jupyter, tag = name) ) 
            if kk == 0:
                a.append(  vi.plot_detectron( img , la , alpha=0.85, 
                                    text_off =True,
                                    draw_bound=False, 
                                    jupyter=show_in_jupyter, tag = name) 
                )
                
            name = f"gt_{i}_frame_{f}"
            gts.append( vi.plot_detectron(  img , ba["label"]+1 , alpha=0.85, 
                                    text_off =True,
                                        draw_bound=False, 
                                        jupyter=show_in_jupyter, tag = name))
            try:
                name = f"aux_{i}_frame_{f}"
                dets.append( vi.plot_detectron(  img , ba["aux_label"]+1 , alpha=0.85, 
                                        text_off =True,
                                            draw_bound=False, 
                                            jupyter=show_in_jupyter, tag = name))
            except:
                pass
        
        ref_pred.append(a)
        if i == kk-1:
            plot_seq( seq )
            plot_seq( gts )
            try:
                plot_seq( ref_pred[i] ) 
            except:
                pass

            try:
                plot_seq( dets )
            except:
                pass
            print("Done")
            print()


In [ ]:
len(seq)